# Языковое моделирование
В данном задании вы реализуете:
- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- EM-algorithm
- Kneser-Ney smoothing

Вы примените это к:
- Language recognition problem

**Подпись**: *Дамдинов Ринчин Гармаевич, ФИТ, 1-й курс магистратуры*

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Возникает закономерный вопрос, а зачем нам это надо? Например, в задачах _машинного перевода_ частенько нужно среди нескольких предложений выбирать наиболее вероятный перевод (который является естественным для человеческого глаза). Также это чрезвычайно полезно в задаче _исправления опечаток_ и _распозновании речи_.

Наша задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **язковой моделью** (LM от Language Model).

Пришло время вспомнить _цепное правило_ (chain rule) $P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1})$. Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения ситуации применим **марковское предположение**: $P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$ для некоторого фиксированного (небольшого) $k$. Это предположение интуитивно ясно и говорит нам о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Пришло время написать класс для хранения n-грам. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [6]:
import re
import math
import random
import numpy as np
from collections import Counter

In [91]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in xrange(self.__max_n + 1)}

        __max = 0        
        ### YOUR CODE HERE   
        number_of_words = 0
        self.add_unk_token()
        
        for sent in sents:
            if( self.__max_n > len(sent)):
                self.__max = len(sent)
            else:
                self.__max = self.__max_n

            for indent in xrange(1, self.__max + 1):
                for begin in xrange(len(sent) + 1 - indent):
                    self.__ngrams[indent][tuple(sent[begin:begin + indent])] += 1

        self.__ngrams[0][()] = len(self.__ngrams[1])
        ### END YOUR CODE
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or u'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
        
    @property
    def max_n(self):
        """Get max_n."""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            print(len(ngram))
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][(u'UNK', )]
        return self.__ngrams[len(ngram)][ngram]

Давайте скачаем корпус и запустим на нем наши эксперименты.

In [4]:
import nltk
# Uncomment next row and download brown corpus
nltk.download()
from nltk.corpus import brown

showing info http://www.nltk.org/nltk_data/


In [7]:
all_sents = list(brown.sents())
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [8]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [92]:
storage = NGramStorage(train_sents, 3)
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('Muammar',)))
print(storage(()))

1338
3282
29
1
2779575


## Оценка качества

Определим **перплексию**:

$$
    {\rm PP}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

кстати, из этой формулы видно, что задача по минимизации перплексии равносильна задаче по максимизации вероятности.

Реализуйте функцию по подсчету перплексии (сначала посмотрите как реализован StraightforwardProbabilityEstimator, а далее уже пишите код).

In [10]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    ### YOUR CODE HEREzz
    # Avoid log(0) by replacing zero by 10 ** (-50).
    N = 0
    logSum = 0;
    for sent in sents:
        N += len(sent)
    logSum = np.sum([math.log(estimator.prob(sent) + 10 ** (-50)) for sent in sents])
    perp = math.exp( -1./N * logSum)
    ### END YOUR CODE
    
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. Что-ж, пора это реализовать.

In [83]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context

    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not (isinstance(word, str) or isinstance(word, unicode)):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in xrange(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [40]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 238.19490166
0.000376909343384
0.0


Посчитаем перплексию униграмной модели.

In [97]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))

Simple estimator perplexity = 38.4472732112


In [98]:
print(storage(tuple('To'.split())))
print(storage(tuple('To be'.split())))
print(storage(tuple('To be or'.split())))


344
17
0


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

* Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?
* Почему перплексия униграмной модели меньше, чем триграмной?

ОТВЕТ:
* Как минимум один ноль дает 'To be or', и UKN токен добавляется только для 1-грам
* Перплексия возрастает с уменьшением вероятности, а вероятность уменьшается с увеличением числа слов в кортеже.

## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Каким-то образом мы хотим избавиться от нулевых вероятностей. Наиболее простой алгоритм борьбы с этой проблемой следующий: для оценки $P(w_{N} \mid w_1^{N - 1})$ мы будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Теперь пришло время снова пописать код. Реализуйте класс, симулирующий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [13]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not (isinstance(word, str) or isinstance(word, unicode)):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        pcounts = self.__storage(context + (word, )) + self.__delta
        ccounts = self.__storage(context) + self.__delta * self.__storage[0][()]

        prob = 1. * pcounts / ccounts
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in xrange(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

In [11]:
avg=0.
for i in range(1, storage .max_n):
    avg+=sum(storage[i].values()) / len(storage[i]) 
avg = avg / (storage.max_n-1)
best_delta = avg/storage[0][()]
print "best delta:", best_delta 

best delta: 0.0001983772739


In [100]:
# Try to find out best delta parametr. We will not provide you any strater code.
### YOUR CODE HERE
import numpy as np
from scipy.optimize import minimize

best_delta = 1

def rosen(x):
    return perplexity( LaplaceProbabilityEstimator(storage, best_delta) , test_sents)

res = minimize(rosen, best_delta, method='nelder-mead', options={'xtol': 1e-8, 'disp': True, 'maxiter': 50})
### END YOUR CODE
best_delta

Optimization terminated successfully.
         Current function value: 144.551659
         Iterations: 4
         Function evaluations: 11


1e-06

In [14]:
# Initialize estimator
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)

# Let's make some estimations
print('Laplace estimator perplexity = {}'.format(perplexity(laplace_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Laplace estimator perplexity = 124.282988583
0.000366017304912


## Stupid backoff

Идея **тупого откатывания** невероятно проста (на то оно и тупое!). Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, тогда будем использовать $k$-грамы. Если эта вероятность крайне мала, то будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не принципиально важно.

Реализуйте класс, симулирующий сглаживание тупым откатыванием. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [84]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = 0
        while(prob == 0):
            prob = self.__base_estimator(word, context)
            context = context[1:]
        prob *= self.__mult
        
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate SCORE of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in xrange(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [103]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(sbackoff_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 64.4132411335
5.3865652725e-05


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

* Почему бессмысленно измерять перплексию в случае **Stupid backoff**?

В SB не получается вероятносное распределение, а для перплексии оно нужно

## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Ну а далее просто постулируем, что

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$. Казалось бы, их нужно несколько, ибо наша модель должна уметь считать вероятности $P(w_3 \mid w_1, w_2)$, а иногда $P(w_2 \mid w_1)$. Если мы тупо обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [101]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        if context == ():
            return self.__base_estimator(word, context)
        
        context = self.__base_estimator.cut_context(context)     
        lambdas = np.zeros(len(context))
                
        extraLambda = np.sum([ self.lambdas[i] for i in xrange(len(context) + 1,len(self.lambdas))]) / len(context)
        prob = np.sum([(self.lambdas[-i] + extraLambda)* self.__base_estimator(word, context[i:]) 
                       for i in range(len(context) + 1)])
        ### END YOUR CODE
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in xrange(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [102]:
import numpy as np

# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 50.4964003997
0.00110501245471


Остается один вопрос: как подбирать $\bar\lambda$? Для этого обычно применяется EM-алгоритм. Сразу опишем лишь готовую формулу. Пусть $\bar\lambda^t$ соответствует набору коэффициентов на $t$-м шаге. $\bar\lambda^0$ задаем произвольно.

* **E-step**:
$$
    \hat\lambda_j^t = \sum_{w_1^N} \frac{\lambda_j^t \cdot \hat P_{S}(w_N \mid w_{N-j+1}^{N-1})}{\sum_{i=1}^N \lambda_i^t \hat P_{S}(w_N \mid w_{N-i+1}^{N-1})}.
$$
* **M-step**:
$$
    \lambda_j^{t+1} = \frac{\hat\lambda_j^t}{\sum_{i=1}^N \hat\lambda_i^t}.
$$

Останавливаться алгоритм должен, когда $\|\hat\lambda_j^t - \hat\lambda_i^{t+1}\| < \varepsilon$.

Формулы выписаны, то бишь самое сложное сделано. Вам остается это лишь реализовать. Документацию для таких формул можете потренироваться написать сами.

In [106]:
def E_step(test_storage, s_estimator, i_estimator):
    ### YOUR CODE HERE
    N = len(i_estimator.lambdas)
    estimated_lambdas = np.zeros(len(i_estimator.lambdas)) 
    
    for i in xrange( 1, test_storage.max_n+1):
        for tlp in list(test_storage[i]):
            denominator = i_estimator(tlp[len(tlp)-1],tlp[:len(tlp)-1])
            if( denominator != 0):
                for j in xrange(len(estimated_lambdas)):
                    numerator = s_estimator(tlp[len(tlp)-1], tlp[ len(tlp)-j : len(tlp)-1 ] )
                    estimated_lambdas[j] += i_estimator.lambdas[j] * numerator / denominator
    ### END YOUR CODE
    
    return estimated_lambdas

def M_step(estimated_lambdas):
    ### YOUR CODE HERE
    new_lambdas = estimated_lambdas / np.sum(estimated_lambdas)
    ### END YOUR CODE
    
    return new_lambdas

def EM_algorithm(test_storage, s_estimator, i_estimator, epsilon=0.03):
    ### YOUR CODE HERE
    old_lambdas = i_estimator.lambdas 
    while True:
        new_lambdas = M_step(E_step(test_storage, s_estimator, i_estimator))
        i_estimator.lambdas = new_lambdas
        if np.linalg.norm(new_lambdas - old_lambdas) < epsilon:
            break
        old_lambdas = new_lambdas#estimated_lambdas
    ### END YOUR CODE
    
    return i_estimator.lambdas

In [107]:
# Separate train into train and test
train_set = train_sents[:int(0.6 * len(train_sents))]
test_set = train_sents[int(0.6 * len(train_sents)):]
MAX_N = 3
train_storage = NGramStorage(train_set, MAX_N)
test_storage = NGramStorage(test_set, MAX_N)
s_estimator = StraightforwardProbabilityEstimator(train_storage)

# Make starting assumption
starting_lambdas = np.array([0.33, 0.33, 0.34])
i_estimator = InterpolationProbabilityEstimator(s_estimator, starting_lambdas)

In [108]:
# It can take some time
best_lambdas = EM_algorithm(test_storage, s_estimator, i_estimator)

In [109]:
best_lambdas

array([ 0.49792053,  0.49792053,  0.00415894])

In [110]:
# Initialize estimator
i_estimator = InterpolationProbabilityEstimator(simple_estimator, best_lambdas)

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(i_estimator, test_sents)))
print(i_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 47.3973683768
0.00017471961809


## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые исползуются в паре-тройке контекстов получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
    N_{1+}(\cdot w_2) := \left|\{w_1 : c(w_1, w_2) > 0\}\right|,
$$
$$
    N_{1+}(\cdot \cdot) := \sum_{w_2} N_{1+}(\cdot w_2),
$$
$$
    \hat P_{KN} (w_1) = \frac{N_{1+}(\cdot w_1)}{N_{1+}(\cdot \cdot)}.
$$

Далее мы используем реккурентное соотношение

$$
    \hat P_{KN}(w_{N} \mid w_1^{N-1}) = \frac{{\rm max}\{c(w_1^N) - \delta, 0\}}{\sum_{w_N}c(w_1^{N-1}w_N)} + \frac{\delta}{\sum_{w_N}c(w_1^{N-1}w_N)}N_{1+}(w_1^{N-1}\cdot)\hat P_{KN}(w_N \mid w_2^{N-1}).
$$

Для вас дело за малым — реализовать это.

In [105]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        self.__postfixs = Counter()
        self.__prefixs = Counter()

        # подсчет числа появлений wi после любого др. слова
        # N+(*w)
        if(self.__storage.max_n > 1):
            for i in self.__storage[2]:
                self.__postfixs[i[-1]] += 1
        # N+(w*)
        for i in range(1, self.__storage.max_n):
            for j in self.__storage[i + 1]:
                self.__prefixs[j[:-1]] += 1

        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not (isinstance(word, str) or isinstance(word, unicode)):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        
        ### YOUR CODE HERE
        #Т.е. если 1-грамма 
        if context == ():
            prob = max([self.__postfixs[word] - self.__delta, 0]) / len(storage[2]) + self.__delta / self.__storage[0][()]
        #если частота контекста 0, берем меньший контекст
        elif (self.__storage(context) == 0):
            prob = self(word, context[1:])
        else:
            prob = self.__prefixs[context] * self.__delta * self(word, context[1:])
            prob += np.max([self.__storage(context + (word, )) - self.__delta, 0])
            prob /= self.__storage(context) 
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in xrange(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [106]:
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 150.975203892
4.29271574498e-06
4.7454835077e-13


## Определение языка документа.

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частотности символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папочке _data_ находятся две папочки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папочек _full_ и _plain_ находятся папочки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папочке находится файл _ans.csv_, в котором вы можете найти правильные ответ и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

In [95]:
import os
from os import walk
import csv

trainFiles = os.path.join(os.getcwd(), 'plain', 'train')
testFiles = os.path.join(os.getcwd(), 'plain', 'test')
plain = os.path.join(os.getcwd(), 'plain')

answers = {}
with open(os.path.join(plain, 'ans.csv'), 'rb') as csvfile:
    reader = csv.reader(csvfile)
    for rows in reader:
        answers[rows[0]] = rows[1]

In [93]:
class LClf():
    
    def __init__(self, n):
        self.__dict = {}
        self.__n = n
        
    def fit(self, fileNames, train_path):
        for filename in fileNames:
            with open(os.path.join(train_path, filename), 'r') as csvfile:
                train_sents = csvfile.readlines()
                train = [ [letter for letter in word] for sent in train_sents for word in sent.split(' ')]
                
                storage = NGramStorage(train, self.__n)
                simple_estimator = StraightforwardProbabilityEstimator(storage)
                sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)
                #StupidBackoffProbabilityEstimator(NGramStorage(train, self.__n),.4)
                self.__dict[filename.split('.')[0]] = sbackoff_estimator
         
    def predict(self, fileNames, test_path):
        res = {}
        for filename in fileNames:
            with open(os.path.join(test_path, filename), 'r') as csvfile:
                test_sents = csvfile.readlines()
                test = [ [letter for letter in word] for sent in test_sents for word in sent.split(' ')]
                
                #res[filename.split('.')[0]] = min(self.__dict.keys(), key = lambda cl: -log(self.__dict[cl]) + \
                #    sum(-log(prob.get((cl,feat), 10**(-7))) for feat in feats))
                
                res[filename.split('.')[0]] = min(self.__dict.iterkeys(), 
                                key=(lambda key: perplexity(self.__dict[key], test)))
            
        return res

In [94]:
def getFiles(path):
    test_texts = []
    for (dirpath, dirnames, filenames) in walk(path):
        test_texts.extend(filenames)
        break
    return test_texts

In [96]:
%%time
clf = LClf(3)
clf.fit(getFiles(trainFiles),trainFiles)

Wall time: 2min 38s


In [97]:
%%time
y_pred = clf.predict(getFiles(testFiles), testFiles)

Wall time: 10min 2s


### На основе языковых моделей, на 3-граммах

In [107]:
resb = np.sum([ y_pred[key] == answers[key] for key in answers.keys()])


In [112]:
print (float(resb) / len(answers))


1.0


In [115]:
print' right %i' % (resb)
print' from %i' % (len(answers))

 right 240
 from 240


In [113]:
%%time
clf = LClf(1)
clf.fit(getFiles(trainFiles),trainFiles)

Wall time: 1min 24s


In [116]:
%%time
y_pred = clf.predict(getFiles(testFiles), testFiles)

Wall time: 6min


###Наивный

In [117]:
resb = np.sum([ y_pred[key] == answers[key] for key in answers.keys()])
print (float(resb) / len(answers))
print' right %i' % (resb)
print' from %i' % (len(answers))

0.9
 right 216
 from 240
